# 코로나 백신 예방접종 현황 API

- 공공데이터활용지원센터
- https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15077756#tab_layer_detail_function

- 보건복지부 접종자 수 질병관리청 접종자 수 비교해서 같으면 사용하기 

- 날짜 / 1차 백신 접종자 수 / 누적 1차 백신 접종자 수 / 2차 백신 접종자 수 / 누적 2차 백신 접종자 수

- baseDate	통계 기준일자

- sido	지역명칭

- firstCnt	당일 통계(1차 접종)

- secondCnt	당일 통계(2차 접종)

- totalFirstCnt	



- **totalSecondCnt	전체 누적 통계(2차 접종)**

- accumulatedFirstCnt	    전일까지의 누적 통계 (1차 접종)

- accumulatedSecondCnt	전일까지의 누적 통계 (2차 접종)

In [1]:
# 한국의 백신 접종자 수 분석
# https://gunn.kim/post/2020-06-13-seoul-metro-data-openapi/
import urllib.request
import json
import pandas as pd 


perPage = '10000'
service_key = 'SAJ%2FFc5PaHo%2FvmwRQl7j9ul5jp43QfUVR9xuEjZMqoL0B6W%2B%2F%2FRR5JDSY1JN%2BDBIhTDheUhdhBZqR%2FGVOQP%2Btg%3D%3D'
date = 'cond[baseDate::EQ]=2021-07-10 00:00:00'
url = 'https://api.odcloud.kr/api/15077756/v1/vaccine-stat?serviceKey='+ service_key + '&' + '&page=1&perPage=' + perPage 

response = urllib.request.urlopen(url)
json_str = response.read().decode('utf-8')
json_object = json.loads(json_str)

df_vac = pd.json_normalize(json_object['data'])

is_korea1 = df_vac['sido'] != '전국'
is_korea2 = df_vac['sido'] != '기타'
df_vac_1 = df_vac[is_korea1 & is_korea2]

In [2]:
df_vac_1['sido'].unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'], dtype=object)

In [3]:
df_vac_1

,accumulatedFirstCnt,accumulatedSecondCnt,accumulatedThirdCnt,baseDate,firstCnt,id,secondCnt,sido,thirdCnt,totalFirstCnt,totalSecondCnt,totalThirdCnt
1,61232,0,0,2021-03-11 00:00:00,12595,2,0,서울특별시,0,73827,0,0
2,36734,0,0,2021-03-11 00:00:00,3311,3,0,부산광역시,0,40045,0,0
3,23101,0,0,2021-03-11 00:00:00,2310,4,0,대구광역시,0,25411,0,0
4,23879,0,0,2021-03-11 00:00:00,1791,5,0,인천광역시,0,25670,0,0
5,20280,0,0,2021-03-11 00:00:00,2403,6,0,광주광역시,0,22683,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6337,1582963,1568183,1190249,2022-02-25 00:00:00,188,6338,327,전라북도,4548,1583151,1568510,1194797
6338,1641848,1625527,1268111,2022-02-25 00:00:00,194,6339,350,전라남도,4106,1642042,1625877,1272217
6339,2279027,2250370,1589927,2022-02-25 00:00:00,410,6340,686,경상북도,7247,2279437,2251056,1597174
6340,2867016,2832654,1965424,2022-02-25 00:00:00,427,6341,778,경상남도,9730,2867443,2833432,1975154


In [8]:
# '1차 백신 접종자 수','누적 1차 백신 접종자 수','2차 백신 접종자 수','누적 2차 백신 접종자 수'
df_vac_2 = df_vac_1[['baseDate', 'sido', 'firstCnt', 'totalFirstCnt' , 'accumulatedFirstCnt', 'secondCnt', 'totalSecondCnt' , 'accumulatedSecondCnt']]

# 날짜 형식 통일
import datetime

for i in range(len(df_vac_2)):
    df_vac_2.iloc[i,0] = datetime.datetime.strptime(df_vac_2.iloc[i,0], '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d')

df_vac_2.columns = ['날짜', '시도명', '당일 통계(1차 접종)','전체 누적 통계(1차 접종)','전일까지의 누적 통계 (1차 접종)', '당일 통계(2차 접종)', '전체 누적 통계(2차 접종)','전일까지의 누적 통계 (2차 접종)']
df_vac_2

# 백신접종시작일인 21년 2월 27일 데이터부터 3월 11일까지의 데이터가 생략되어 있음. 후에 추가로 생성 예정

,날짜,시도명,당일 통계(1차 접종),전체 누적 통계(1차 접종),전일까지의 누적 통계 (1차 접종),당일 통계(2차 접종),전체 누적 통계(2차 접종),전일까지의 누적 통계 (2차 접종)
1,2021-03-11,서울특별시,12595,73827,61232,0,0,0
2,2021-03-11,부산광역시,3311,40045,36734,0,0,0
3,2021-03-11,대구광역시,2310,25411,23101,0,0,0
4,2021-03-11,인천광역시,1791,25670,23879,0,0,0
5,2021-03-11,광주광역시,2403,22683,20280,0,0,0
...,...,...,...,...,...,...,...,...
6337,2022-02-25,전라북도,188,1583151,1582963,327,1568510,1568183
6338,2022-02-25,전라남도,194,1642042,1641848,350,1625877,1625527
6339,2022-02-25,경상북도,410,2279437,2279027,686,2251056,2250370
6340,2022-02-25,경상남도,427,2867443,2867016,778,2833432,2832654


In [9]:
df_vac_2[df_vac_2['시도명'] == '서울특별시']

,날짜,시도명,당일 통계(1차 접종),전체 누적 통계(1차 접종),전일까지의 누적 통계 (1차 접종),당일 통계(2차 접종),전체 누적 통계(2차 접종),전일까지의 누적 통계 (2차 접종)
1,2021-03-11,서울특별시,12595,73827,61232,0,0,0
19,2021-03-12,서울특별시,10972,85864,74892,0,0,0
37,2021-03-13,서울특별시,9654,95996,86342,0,0,0
55,2021-03-14,서울특별시,1704,97777,96073,0,0,0
73,2021-03-15,서울특별시,900,98677,97777,0,0,0
...,...,...,...,...,...,...,...,...
6253,2022-02-21,서울특별시,50,8290271,8290221,100,8201009,8200909
6271,2022-02-22,서울특별시,1361,8293370,8292009,860,8203591,8202731
6289,2022-02-23,서울특별시,1035,8295316,8294281,628,8205164,8204536
6307,2022-02-24,서울특별시,929,8296984,8296055,570,8206487,8205917


## 시도명 으로 묶기

In [11]:
# df_conf = df_conf[''].groupby(df_conf[''])
grouped = df_vac_2.groupby(['시도명', '날짜'], as_index=False).sum()
grouped

,시도명,날짜,당일 통계(1차 접종),전체 누적 통계(1차 접종),전일까지의 누적 통계 (1차 접종),당일 통계(2차 접종),전체 누적 통계(2차 접종),전일까지의 누적 통계 (2차 접종)
0,강원도,2021-03-11,2627,14790,12163,0,0,0
1,강원도,2021-03-12,2314,17144,14830,0,0,0
2,강원도,2021-03-13,1336,18481,17145,0,0,0
3,강원도,2021-03-14,72,18553,18481,0,0,0
4,강원도,2021-03-15,0,18553,18553,0,0,0
...,...,...,...,...,...,...,...,...
5979,충청북도,2022-02-21,4,1418181,1418177,18,1403400,1403382
5980,충청북도,2022-02-22,253,1418472,1418219,176,1403615,1403439
5981,충청북도,2022-02-23,241,1418757,1418516,130,1403796,1403666
5982,충청북도,2022-02-24,180,1418969,1418789,104,1403932,1403828


In [12]:
# 현재 날짜에 맞춰 '2021년  N월 M주차_국내 코로나 확진자-백신접종자 현황'으로 저장하기
import datetime as dt
from datetime import datetime, timedelta

def get_date(y, m, d):
    s = f'{y:04d}-{m:02d}-{d:02d}'
    return datetime.strptime(s, '%Y-%m-%d')

def get_week_no(y, m, d):
    target = get_date(y, m, d)
    firstday = target.replace(day = 1)
    if firstday.weekday() == 6:
        origin = firstday
    elif firstday.weekday() < 3:
        origin = firstday - timedelta(days=firstday.weekday() + 1)
    else:
        origin = firstday + timedelta(days = 6-firstday.weekday())
    return (target - origin).days // 7 + 1

# wlsxwriter 라이브러리를 사용해 데이터를 시트별로 저장
import xlsxwriter
import datetime as dt

n = dt.datetime.now()
writer = pd.ExcelWriter('국내 코로나 확진자-백신접종자 지역별 일일 현황_{}년 {}월 {}주차.xlsx'
                        .format(n.year, n.month, get_week_no(n.year, n.month, n.day)), 
                        engine = 'xlsxwriter')

grouped.to_excel(writer, index = False)

writer.save()

## 일일 확진자 도시별로 시트별로 저장

In [71]:
# 유일한 값 찾기
city_list = grouped['시도명'].unique().tolist()
city_list

['강원도',
 '경기도',
 '경상남도',
 '경상북도',
 '광주광역시',
 '대구광역시',
 '대전광역시',
 '부산광역시',
 '서울특별시',
 '세종특별자치시',
 '울산광역시',
 '인천광역시',
 '전라남도',
 '전라북도',
 '제주특별자치도',
 '충청남도',
 '충청북도']

In [72]:
df_list = []

for city in city_list:
    df_list.append(grouped[grouped['시도명']==city])

In [73]:
df_list

[     시도명        날짜  당일 통계(1차 접종)   전체 누적 통계(1차 접종)  전일까지의 누적 통계 (1차 접종)  \
 0    강원도  20210311          2627             14790                12163   
 1    강원도  20210312          2314             17144                14830   
 2    강원도  20210313          1336             18481                17145   
 3    강원도  20210314            72             18553                18481   
 4    강원도  20210315             0             18553                18553   
 ..   ...       ...           ...               ...                  ...   
 347  강원도  20220221             4           1339853              1339849   
 348  강원도  20220222           215           1340095              1339880   
 349  강원도  20220223           206           1340394              1340188   
 350  강원도  20220224           166           1340633              1340467   
 351  강원도  20220225           181           1340887              1340706   
 
      당일 통계(2차 접종)   전체 누적 통계(2차 접종)  전일까지의 누적 통계 (2차 접종)  
 0               0        

In [74]:
import datetime as dt
from datetime import datetime, timedelta

def get_date(y, m, d):
    s = f'{y:04d}-{m:02d}-{d:02d}'
    return datetime.strptime(s, '%Y-%m-%d')

def get_week_no(y, m, d):
    target = get_date(y, m, d)
    firstday = target.replace(day = 1)
    if firstday.weekday() == 6:
        origin = firstday
    elif firstday.weekday() < 3:
        origin = firstday - timedelta(days=firstday.weekday() + 1)
    else:
        origin = firstday + timedelta(days = 6-firstday.weekday())
    return (target - origin).days // 7 + 1

In [75]:
n = dt.datetime.now()

writer = pd.ExcelWriter('국내 코로나 확진자-백신접종자 지역별 일일 현황_{}년 {}월 {}주차.xlsx'
                        .format(n.year, n.month, get_week_no(n.year, n.month, n.day)), 
                        engine = 'xlsxwriter')

In [76]:
for i, df in enumerate(df_list):
    print(city_list[i])
    print(df)
    df.to_excel(writer, index = False, sheet_name = city_list[i])
    
writer.save()

강원도
     시도명        날짜  당일 통계(1차 접종)   전체 누적 통계(1차 접종)  전일까지의 누적 통계 (1차 접종)  \
0    강원도  20210311          2627             14790                12163   
1    강원도  20210312          2314             17144                14830   
2    강원도  20210313          1336             18481                17145   
3    강원도  20210314            72             18553                18481   
4    강원도  20210315             0             18553                18553   
..   ...       ...           ...               ...                  ...   
347  강원도  20220221             4           1339853              1339849   
348  강원도  20220222           215           1340095              1339880   
349  강원도  20220223           206           1340394              1340188   
350  강원도  20220224           166           1340633              1340467   
351  강원도  20220225           181           1340887              1340706   

     당일 통계(2차 접종)   전체 누적 통계(2차 접종)  전일까지의 누적 통계 (2차 접종)  
0               0                 0 

대전광역시
        시도명        날짜  당일 통계(1차 접종)   전체 누적 통계(1차 접종)  전일까지의 누적 통계 (1차 접종)  \
2112  대전광역시  20210311          1952             14525                12573   
2113  대전광역시  20210312          1482             16010                14528   
2114  대전광역시  20210313          1210             17283                16073   
2115  대전광역시  20210314            66             17349                17283   
2116  대전광역시  20210315             0             17349                17349   
...     ...       ...           ...               ...                  ...   
2459  대전광역시  20220221             5           1238639              1238634   
2460  대전광역시  20220222           243           1238975              1238732   
2461  대전광역시  20220223           177           1239208              1239031   
2462  대전광역시  20220224           177           1239450              1239273   
2463  대전광역시  20220225           175           1239693              1239518   

      당일 통계(2차 접종)   전체 누적 통계(2차 접종)  전일까지의 누적 통계 (2차 접종)

[352 rows x 8 columns]
전라남도
       시도명        날짜  당일 통계(1차 접종)   전체 누적 통계(1차 접종)  전일까지의 누적 통계 (1차 접종)  \
4224  전라남도  20210311          2718             25748                23030   
4225  전라남도  20210312          2202             28293                26091   
4226  전라남도  20210313          1608             29987                28379   
4227  전라남도  20210314            41             30195                30154   
4228  전라남도  20210315             0             30195                30195   
...    ...       ...           ...               ...                  ...   
4571  전라남도  20220221             2           1641119              1641117   
4572  전라남도  20220222           261           1641371              1641110   
4573  전라남도  20220223           180           1641612              1641432   
4574  전라남도  20220224           214           1641825              1641611   
4575  전라남도  20220225           194           1642042              1641848   

      당일 통계(2차 접종)   전체 누적 통계(2차 접종)  전일까지의 누적 